In [1]:
%pip install pandas torch transformers beir scikit-learn scipy faiss-cpu tensorboard pyarrow fastparquet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.7 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 31.2 MB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 44.4 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 80.1 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 55.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 38.5 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 80.6 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 80.4 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 79.5 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

CUDA available: True
GPU: NVIDIA A30
GPU Memory: 25.2 GB


In [3]:
#Tokenize data
#Place datasets under contriever/scripts/preprocess/
!bash ./scripts/preprocess/tokenization_script.sh

./wikipedia_en_20231101.txt
./scripts/preprocess/tokenization_script.sh: line 14: ./wikipedia_en_20231101.txt: No such file or directory
Splitting  lines into chunks of 0 lines...
tokenizer_config.json: 100%|██████████████████| 48.0/48.0 [00:00<00:00, 561kB/s]
config.json: 100%|█████████████████████████████| 570/570 [00:00<00:00, 8.92MB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 4.95MB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 8.92MB/s]
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101.txt/000...
Saving at ./encoded-data/bert-base-uncased/wikipedia_en_20231101/000.pkl...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101.txt/007...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101.txt/001...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101.txt/002...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101.txt/004...
Encoding ./tmp-tokenizati

In [10]:
# training the data
# prof mentioned that you should be able to hit 70% accuracy just by training on more data (TA trained for >= 1 day)
!bash /users/vishraj/cs511-fall2025-p3-contriever/scripts/train/contriever.sh

Using device: cuda
                            augmentation: delete                                  	[default: none]
                                   beta1: 0.9                                     
                                   beta2: 0.98                                    
                            chunk_length: 256                                     
                       continue_training: False                                   
                        contrastive_mode: moco                                    
                                 dropout: 0.1                                     
                                     eps: 1e-06                                   
                               eval_data: []                                      
                           eval_datasets: []                                      
                       eval_datasets_dir: ./                                      
                               eval_freq: 500       

In [ ]:
# if you need to delete a model checkpoint
# !rm -rf ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1

In [2]:
# Evaluate your model using NQ benchmark
#Step1: prepare passage subset
#run under contriever/scripts/evaluation/FiD/open_domain_data
#place psgs_w100.tsv under contriever/scripts/evaluation/FiD/open_domain_data/
# !cp drive/MyDrive/datasets/psgs_w100.tsv drive/MyDrive/contriever/scripts/evaluation/FiD/open_domain_data/

!python3 ./scripts/evaluation/FiD/open_domain_data/select_question_subset.py \
--data_dir ./scripts/evaluation/FiD/open_domain_data/download > ./scripts/evaluation/FiD/open_domain_data/passage.idx

In [7]:
!python3 ./scripts/evaluation/FiD/open_domain_data/create_passage_subset.py \
--passage_idx ./scripts/evaluation/FiD/open_domain_data/passage.idx \
--passages_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100.tsv \
--output_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv \
--seed 60

Loading required passage IDs from ./scripts/evaluation/FiD/open_domain_data/passage.idx...
Loaded 92042 required passage IDs

Target subset size: 210153 passages (1% of 21015324)
Scanning passages file (byte-preserving)...
Reading lines: 100%|█████████| 21015324/21015324 [00:28<00:00, 745056.38lines/s]

Required passages found: 92042
Additional random passages needed: 118111

SUBSET SUMMARY
Total passages in original: 21,015,324
Required IDs: 92,042
Required passages found: 92,042
Final subset size: 210,153
Target (1%): 210,153
Percentage of original: 1.00%

Saving subset to ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv (byte-identical lines)...
✓ Saved 210153 lines to ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv


In [11]:
#Evaluate your model using NQ benchmark
#Step2: preprocess for testing
# !python3 ./scripts/evaluation/FiD/src/preprocess.py \
# --num_questions 1000 --passages_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv

!python3 ./scripts/evaluation/FiD/src/preprocess.py \
--num_questions 10000 --passages_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv


Loading passages from ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv
Loading passages: 100%|█████████████| 210154/210154 [00:01<00:00, 120476.15it/s]
Loaded 210153 passages
Selecting test examples
Traceback (most recent call last):
  File "/users/vishraj/cs511-fall2025-p3-contriever/./scripts/evaluation/FiD/src/preprocess.py", line 67, in <module>
    NQ_test = select_examples_NQ(originaldev, NQ_idx['test'][:num_questions], passages, NQ_passages['test'])
  File "/users/vishraj/cs511-fall2025-p3-contriever/./scripts/evaluation/FiD/src/preprocess.py", line 18, in select_examples_NQ
    ctxs = [
  File "/users/vishraj/cs511-fall2025-p3-contriever/./scripts/evaluation/FiD/src/preprocess.py", line 21, in <listcomp>
    'title': passages[idx][1],
KeyError: '5520085'


In [9]:
# Step3: use your model to embed passages
!python3 ./scripts/evaluation/generate_passage_embeddings.py  \
--passages ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv \
 --model_name_or_path ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/step-100000 \
 --output_dir ./scripts/evaluation/contriever_embeddings  \
 --shard_id 0 \
 --num_shards 1

Using device: cuda
Model loaded from ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/step-100000.
Embedding generation for 210153 passages from idx 0 to 210153.
Encoding passages: 100%|█████████| 210153/210153 [02:51<00:00, 1228.29passage/s]
Saving 210153 passage embeddings to ./scripts/evaluation/contriever_embeddings/passages_00.
Total passages processed 210153. Written to ./scripts/evaluation/contriever_embeddings/passages_00.


In [12]:
# Step4: retrieve the passages
#run under scripts/evaluation/FiD/open_domain_data
!python3 ./scripts/evaluation/passage_retrieval.py \
--model_name_or_path ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/step-100000  \
--passages ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv  \
--passages_embeddings ./scripts/evaluation/contriever_embeddings/*  \
--data ./open_domain_data/NQ/test.json \
--output_dir ./scripts/evaluation/FiD/open_domain_data/contriever_nq

Using device: cuda
Loading model from: ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/step-100000
Indexing passages from files ['./scripts/evaluation/contriever_embeddings/passages_00']
Loading file ./scripts/evaluation/contriever_embeddings/passages_00
Total data indexed 210153
Data indexing completed.
Indexing time: 0.9 s.
loading passages
210153
processing path ./open_domain_data/NQ/test.json
Questions embeddings shape: torch.Size([1000, 768])
Searching for 100 documents in the index
100%|███████████████████████████████████████| 1000/1000 [00:35<00:00, 28.36it/s]
Search time: 35.3 s.
Validation results: top k documents hits %s [196, 304, 397, 455, 505, 536, 567, 590, 617, 629, 647, 658, 669, 680, 692, 703, 710, 716, 723, 727, 733, 739, 746, 749, 754, 756, 761, 762, 765, 770, 772, 778, 782, 787, 790, 791, 794, 795, 795, 795, 796, 799, 800, 800, 802, 805, 805, 806, 809, 810, 810, 812, 815, 815, 817, 818, 820, 821, 823, 823, 823, 824, 826, 82